# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import datetime
from sql_queries import *

In [2]:
# to run create_tables.py
%run create_tables.py

In [3]:
# to run etl.py
%run etl.py

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data/')
len(song_files)

71

In [7]:
# select first song on list
filepath = song_files[0]

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
# convert array to list
df.values[0].tolist()

['ARMAC4T1187FB3FA4C',
 40.82624,
 'Morris Plains, NJ',
 -74.47995,
 'The Dillinger Escape Plan',
 207.77751,
 1,
 'SOBBUGU12A8C13E95D',
 'Setting Fire to Sleeping Giants',
 2004]

In [10]:
# select specific columns
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOBBUGU12A8C13E95D',
 'Setting Fire to Sleeping Giants',
 'ARMAC4T1187FB3FA4C',
 2004,
 207.77751]

In [11]:
song_data = (song_data[0], song_data[1], song_data[2], song_data[3], song_data[4])
song_data

('SOBBUGU12A8C13E95D',
 'Setting Fire to Sleeping Giants',
 'ARMAC4T1187FB3FA4C',
 2004,
 207.77751)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data = (artist_data[0], artist_data[1], artist_data[2], artist_data[3], artist_data[4])
artist_data

('ARMAC4T1187FB3FA4C',
 'The Dillinger Escape Plan',
 'Morris Plains, NJ',
 40.82624,
 -74.47995)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files("data/log_data/")

In [16]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-12-events.json'

In [17]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Celeste,F,0,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,438,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,None,Logged In,Jacqueline,F,0,Lynch,NaN,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1.540224e+12,389,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df = df[df['page']=="NextSong"].reset_index()
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
1,2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
2,3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


In [19]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-12 02:37:38.796
1   2018-11-12 02:37:44.796
2   2018-11-12 02:42:21.796
3   2018-11-12 02:45:52.796
4   2018-11-12 02:47:22.796
Name: ts, dtype: datetime64[ns]

In [20]:
df['week'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).isocalendar()[1])
df['week_day'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).strftime("%A"))
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,...,page,registration,sessionId,song,status,ts,userAgent,userId,week,week_day
0,1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",...,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,46,Monday
1,2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",...,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,46,Monday
2,3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",...,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,46,Monday
3,5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",...,NextSong,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,46,Monday
4,6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",...,NextSong,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29,46,Monday


In [21]:
time_data = (t, t.dt.hour, t.dt.day, df.week, t.dt.month, t.dt.year, df.week_day)
column_labels = ['start_time','hour','day','week','month', 'year','weekday']

In [22]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()
df['start_time'] = t
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,...,registration,sessionId,song,status,ts,userAgent,userId,week,week_day,start_time
0,1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",...,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,46,Monday,2018-11-12 02:37:38.796
1,2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",...,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,46,Monday,2018-11-12 02:37:44.796
2,3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",...,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,46,Monday,2018-11-12 02:42:21.796
3,5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",...,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,46,Monday,2018-11-12 02:45:52.796
4,6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",...,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29,46,Monday,2018-11-12 02:47:22.796


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[['userId','firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,10,Sylvie,Cruz,F,free
1,53,Celeste,Williams,F,free
2,53,Celeste,Williams,F,free
3,53,Celeste,Williams,F,free
4,29,Jacqueline,Lynch,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
song_select

'SELECT songs.song_id, artists.artist_id FROM songs \n                  JOIN artists ON  songs.artist_id=artists.artist_id\n                  WHERE songs.title=%s AND artists.name=%s AND songs.duration=%s;'

In [27]:
songplay_table_insert

'INSERT INTO songplays( start_time, user_id,level, artist_id, song_id, session_id, location, user_agent)\n                            VALUES(%s, %s, %s, %s, %s, %s, %s, %s)'

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (str(row.start_time), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.